# Khám phá mối quan hệ trong dữ liệu

## Import các thư viện cần thiết

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np
import statsmodels.api as sm
#from wordcloud import WordCloud
#from statsmodels.tsa.arima.model import ARIMA

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None)

## Đọc dữ liệu từ file csv vào dataframe


In [17]:
data = pd.read_csv("../datasets/data_preprocess.csv", index_col = 0)
data.head()

,film_code,cinema_code,total_sales,tickets_sold,tickets_out,show_time,occu_perc,ticket_price,ticket_use,capacity,date,month,quarter,day
0,1492,304,3900000,26,0,4,4.26,150000.0,26,610.328638,2018-05-05,5,2,5
1,1492,352,3360000,42,0,5,8.08,80000.0,42,519.801980,2018-05-05,5,2,5
2,1492,489,2560000,32,0,4,20.00,80000.0,32,160.000000,2018-05-05,5,2,5
3,1492,429,1200000,12,0,1,11.01,100000.0,12,108.991826,2018-05-05,5,2,5
4,1492,524,1200000,15,0,3,16.67,80000.0,15,89.982004,2018-05-05,5,2,5


## Thêm một số cột cần thiết

### Thêm cột weekday

Cột weekday có giá trị ý nghĩa cho ngày thứ mấy trong tuần

In [18]:
data['date'] = pd.to_datetime(data['date'])

data['weekday'] = data['date'].dt.weekday

### Thêm cột date_from_start

Cột date_from_start có giá trị là số ngày cách biệt kể từ ngày hôm đó đến ngày đầu tiên phim đó được công chiếu.

In [19]:
data_min_date = data.groupby('film_code')['date'].min()
data['date_from_start'] = data.apply(lambda row: (row['date']- data_min_date[row['film_code']]).days, axis=1)


### Tổng vé bán theo phim theo thời gian

**Trường dữ liệu**
- `tickets_sold`, `date`, `film_code`

**Lý do chọn biểu đồ**
- Biểu đồ line phù hợp biểu diễn cho dữ liệu dạng timeseries

**Lý do chọn màu sắc**
- Mỗi film_code 1 màu khác nhau, để dễ phân biệt

In [20]:
data_film = data.groupby(["film_code", "date"])["tickets_sold"].sum().reset_index()
data_film = data_film.sort_values(by=['film_code', 'date'], ascending=[True, True])

In [21]:
fig = px.line(data_film, x='date', y='tickets_sold', color='film_code', title='Tổng vé bán theo phim')
fig.show()

**Nhận xét**
- Dữ liệu có dạng răng cưa, chứng tỏ 2 ngày liền kề thì sẽ chênh lệch nhau rất nhiều. Chứng tỏ phim thường không được chiếu liên tục trong tuần mà nhiều nơi sẽ chiếu theo lịch 2 ngày 1 lần.
- Nết xét trong 1 đoạn thời gian ngắn, dữ liệu có tính chu kỳ (thấy rõ nhất ở đoạn tháng 7 - 9). Chu kỳ này đó chính là theo tuần, tại vì lịch chiếu phim thường phân bố các ngày cố định trong tuần (chẳng hạn thứ 3 6 cn).
- Điểm thú vị đó chính là mọi bộ phim đều thường có đỉnh răng cưa và đáy răng cưa ở cùng 1 ngày => Những phim hay thì luôn được chiếu vào 1 ngày cố định => Muốn xem phim hay thì hãy ra rạp vào những ngày cố định này.
- Xét tổng thể, những phim nổi sẽ chênh lệch rất nhiều so với các phim khác (ví dụ phim 1554).
- Một phim nổi tiếng và có số vé bán cao sẽ là niềm bất hạnh của các phim khác chiếu trong cùng thời gian. Thời gian từ tháng 7-9, phim 1554 chiếm hầu hết các vé bán, các phim còn lại rất ít vé bán. Trong khi ở những khoảng thời gian còn lại, có nhiều phim cùng đạt số lượng vé cao.


### Tổng vé bán theo thời gian kể từ ngày công chiếu

**Trường dữ liệu**
- `tickets_sold`, `date_from_start`, `film_code`

**Lý do chọn biểu đồ**
- Biểu đồ line phù hợp biểu diễn cho dữ liệu dạng timeseries

**Lý do chọn màu sắc**
- Mỗi film_code 1 màu khác nhau, để dễ phân biệt

In [22]:
data_occu_perc = data.groupby(["film_code", "date_from_start"])["tickets_sold"].mean().reset_index()
data_occu_perc = data_occu_perc.sort_values(by=['film_code', 'date_from_start'], ascending=[True, False])

In [23]:
fig = px.line(data_occu_perc, x='date_from_start', y='tickets_sold', color='film_code', 
              title='Tổng vé bán theo phim kể từ ngày công chiếu')
fig.show()


**Nhận xét**
- Biểu đồ cho thấy được số vé bán của mỗi bộ phim thay đổi như thế nào khi càng xa ngày công chiếu
- Theo càng xa ngày công chiếu, mỗi số vé bán của mỗi bộ phim có xu hướng giảm dần.
- Biểu đồ có dạng răng cưa và có tính chu kỳ, điều này là do mỗi bộ phim thường được công chiếu vào một số ngày cố định trong tuần.


### Tỷ lệ chiếm rạp theo phim

**Trường dữ liệu**
- `occu_perc`, `date`, `film_code`

**Lý do chọn biểu đồ**
- Biểu đồ line phù hợp biểu diễn cho dữ liệu dạng timeseries

**Lý do chọn màu sắc**
- Mỗi film_code 1 màu khác nhau, để dễ phân biệt

In [24]:
data_film = data.groupby(["film_code", "date"])["occu_perc"].mean().reset_index()
data_film = data_film.sort_values(by=['film_code', 'date'], ascending=[True, False])

In [25]:
fig = px.line(data_film, x='date', y='occu_perc', color='film_code', title='Tỷ lệ chiếm rạp theo phim')
fig.show()

**Nhận xét**
- Tỷ lệ chiếm rạp không chênh lệch nhiều, nhưng tổng số vé bán thì lại chênh lệch rất nhiều. Chứng tỏ không phải mỗi rạp phim đều chiếu các loại phim như nhau, phim nào hay thì sẽ được chiếu ở nhiều rạp. 
-  Cũng như tổng số vé bán, tỷ lệ chiếm rạp cũng có dạng răng cưa, và nếu xét trong 1 khoảng thời gian ngắn thì nó có chu kỳ.
- Tại mỗi thời điểm, luôn có nhiều phim có tỷ lệ bao rạp cao. Khác với tổng số vé, khi một phim số vé cao thì sẽ ảnh hưởng các phim còn lại, tỷ lệ bao rạp thì sẽ không. Nó được điều chỉnh để cao nhất có thể, khi phim đó ít nổi, thì nó sẽ ít được chiếu lại. Một số bộ phim có doanh thu thấp so với mặt bằng chung nhưng có tỷ lệ chiếm rạp cao là 1496, 1572, 1578, 1492

### Tỷ lệ chiếm rạp theo phim kể từ ngày công chiếu

**Trường dữ liệu**
- `occu_perc`, `date_from_start`, `film_code`

**Lý do chọn biểu đồ**
- Biểu đồ line phù hợp biểu diễn cho dữ liệu dạng timeseries

**Lý do chọn màu sắc**
- Mỗi film_code 1 màu khác nhau, để dễ phân biệt

In [26]:
data_occu_perc = data.groupby(["film_code", "date_from_start"])["occu_perc"].mean().reset_index()
data_occu_perc = data_occu_perc.sort_values(by=['film_code', 'date_from_start'], ascending=[True, False])

In [27]:
fig = px.line(data_occu_perc, x='date_from_start', y='occu_perc', color='film_code', 
              title='Tỷ lệ chiếm rạp theo phim kể từ ngày công chiếu')
fig.show()


**Nhận xét**
- Một bộ phim sẽ không vì được chiếu lâu rồi mà tỷ lệ chiếm rạp bị giảm, tỷ lệ này không có giảm theo thời gian mà khác nhau theo từng bộ phim
- Nhận thấy mỗi bộ phim được chiếu lâu ngày thì số vé bán sẽ giảm nhưng mà rất nhiều bộ phim có tỷ lệ chiếm rạp lại tăng. Như vậy bộ phim chiếu càng lâu thì số rạp chiếu phim đó càng ít.
